In [1]:
from GenZ import get_model_df, get_summary_table, System, create_inference_moe_prefix_model, create_inference_moe_decode_model
import pandas as pd


### LLama7B Dense LLM Prefill Stage

In [2]:
TPU = System(flops=300, offchip_mem_bw=1200, compute_efficiency=0.8, memory_efficiency=0.8, bits='bf16')

In [3]:
LLM_prefill = create_inference_moe_prefix_model(1024, "llama_7b")
model_df = get_model_df(model=LLM_prefill, system=TPU)
model_df

,Op Type,Dimension,Bound,C/M ratio,Op Intensity,Latency (msec),Cycles,C Effcy,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Throughput (Tflops),Compute time (msec),Memory time (msec),Compute cycle,Memory cycle
0,GEMM,"[1, 12288, 1024, 4096]",C,3.298535,768.0,0.429497,403726.925824,0.8,103079.215104,8.0,96.0,24.0,128.0,240.0,0.429497,0.130208,403726.925824,122395.833333
1,Logit,"[1, 32, 1024, 1024, 128, 32]",M,0.439805,102.4,0.08138,76497.395833,0.8,8589.934592,8.0,8.0,64.0,80.0,105.553116,0.035791,0.08138,33643.910485,76497.395833
2,Attend,"[1, 32, 1024, 1024, 128, 32]",M,0.439805,102.4,0.08138,76497.395833,0.8,8589.934592,64.0,8.0,8.0,80.0,105.553116,0.035791,0.08138,33643.910485,76497.395833
3,GEMM,"[1, 4096, 1024, 4096]",C,2.932031,682.666667,0.143166,134575.641941,0.8,34359.738368,8.0,32.0,8.0,48.0,240.0,0.143166,0.048828,134575.641941,45898.4375
4,GEMM,"[1, 22016, 1024, 4096]",C,3.392215,789.811659,0.769515,723344.075435,0.8,184683.593728,8.0,172.0,43.0,223.0,240.0,0.769515,0.226847,723344.075435,213236.490885
5,GEMM,"[1, 4096, 1024, 11008]",C,3.274736,762.458874,0.384757,361672.037717,0.8,92341.796864,21.5,86.0,8.0,115.5,240.0,0.384757,0.117493,361672.037717,110443.115234


In [4]:
llama_summary = get_summary_table(model_df, system=TPU)
llama_summary

,MACs (MFLOP),Total Data (MB),Total Weights (MB),Parameters (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles
0,431644.213248,674.5,386.0,386.0,223.0,1.889695,1.776313e+06


### Llama 7B Dense LLM Decode Stage

In [7]:
LLM_decode = create_inference_moe_decode_model(1024, "llama_7b")
model_df = get_model_df(model=LLM_decode, system=TPU)
model_df

,Op Type,Dimension,Bound,C/M ratio,Op Intensity,Latency (msec),Cycles,C Effcy,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Throughput (Tflops),Compute time (msec),Memory time (msec),Compute cycle,Memory cycle
0,GEMM,"[1, 12288, 1, 4096]",M,0.004294,0.999675,0.097688,91826.756795,0.8,100.663296,0.007812,96.0,0.023438,96.03125,1.030457,0.000419,0.097688,394.264576,91826.756795
1,Logit,"[1, 32, 1, 1024, 128, 32]",M,0.004258,0.991288,0.00821,7716.973623,0.8,8.388608,0.007812,8.0,0.0625,8.070312,1.021811,0.000035,0.00821,32.855381,7716.973623
2,Logit,"[1, 32, 1, 32, 128, 32]",M,0.004134,0.962406,0.000264,248.392423,0.8,0.262144,0.007812,0.25,0.001953,0.259766,0.992041,0.000001,0.000264,1.026731,248.392423
3,Attend,"[1, 32, 1, 1024, 128, 32]",M,0.004258,0.991288,0.00821,7716.973623,0.8,8.388608,0.0625,8.0,0.007812,8.070312,1.021811,0.000035,0.00821,32.855381,7716.973623
4,Attend,"[1, 32, 1, 32, 128, 32]",M,0.004134,0.962406,0.000264,248.392423,0.8,0.262144,0.001953,0.25,0.007812,0.259766,0.992041,0.000001,0.000264,1.026731,248.392423
5,GEMM,"[1, 4096, 1, 4096]",M,0.004293,0.999512,0.032568,30613.899231,0.8,33.554432,0.007812,32.0,0.007812,32.015625,1.030289,0.00014,0.032568,131.421525,30613.899231
6,GEMM,"[1, 22016, 1, 4096]",M,0.004294,0.999711,0.175018,164517.025153,0.8,180.355072,0.007812,172.0,0.041992,172.049805,1.030494,0.000751,0.175018,706.390699,164517.025153
7,GEMM,"[1, 4096, 1, 11008]",M,0.004294,0.999665,0.087513,82262.247801,0.8,90.177536,0.020996,86.0,0.007812,86.028809,1.030447,0.000376,0.087513,353.195349,82262.247801


In [10]:
model_df.to_csv("llama_7b_decode.csv", index=False)

In [8]:
llama_summary = get_summary_table(model_df, system=TPU)
llama_summary

,MACs (MFLOP),Total Data (MB),Total Weights (MB),Parameters (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles
0,422.05184,402.785645,386.0,386.0,172.049805,0.409735,385150.661071


### Mixtral 8x7B MoE LLM Prefill Stage

In [2]:
GH200 = System(flops=2000, offchip_mem_bw=4900, compute_efficiency=0.8, memory_efficiency=0.8, bits='bf16',
            off_chip_mem_size=144)

In [4]:
LLM_prefill = create_inference_moe_prefix_model(1024, "mixtral_8x7b")
model_df = get_model_df(model=LLM_prefill, system=GH200)
model_df

,Op Type,Dimension,Bound,C/M ratio,Op Intensity,Latency (msec),Cycles,C Effcy,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Throughput (Tflops),Compute time (msec),Memory time (msec),Compute cycle,Memory cycle
0,GEMM,"[1, 6144, 1024, 4096]",C,1.901508,722.823529,0.032212,30279.519437,0.8,51539.607552,8.0,48.0,12.0,68.0,1600.0,0.032212,0.01694,30279.519437,15923.947704
1,Logit,"[1, 32, 1024, 1024, 128, 8]",M,0.291222,110.702703,0.018435,17329.001913,0.8,8589.934592,8.0,2.0,64.0,74.0,465.955198,0.005369,0.018435,5046.586573,17329.001913
2,Attend,"[1, 32, 1024, 1024, 128, 8]",M,0.291222,110.702703,0.018435,17329.001913,0.8,8589.934592,64.0,2.0,8.0,74.0,465.955198,0.005369,0.018435,5046.586573,17329.001913
3,GEMM,"[1, 4096, 1024, 4096]",C,1.795869,682.666667,0.021475,20186.346291,0.8,34359.738368,8.0,32.0,8.0,48.0,1600.0,0.021475,0.011958,20186.346291,11240.433673
4,GEMM,"[1, 229376, 256, 4096]",M,0.633171,240.688353,0.474829,446338.887117,0.8,481036.337152,2.0,1792.0,112.0,1906.0,1013.073631,0.300648,0.474829,282608.848077,446338.887117
5,GEMM,"[1, 4096, 256, 114688]",M,0.632507,240.436059,0.237663,223403.61926,0.8,240518.168576,56.0,896.0,2.0,954.0,1012.011709,0.150324,0.237663,141304.424038,223403.61926


In [6]:
llama_summary = get_summary_table(model_df, system=GH200)
llama_summary

,MACs (MFLOP),Total Data (MB),Total Weights (MB),Parameters (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles
0,824633.720832,3124.0,2768.0,2768.0,1906.0,0.803049,754866.375932


In [ ]:
model_df.to_csv("mixtral_8x7b_prefill_on_GH200.csv", index=False)

### Llama 7B Dense LLM Decode Stage

In [ ]:
LLM_decode = create_inference_moe_decode_model(1024, "llama_7b")
model_df = get_model_df(model=LLM_decode, system=TPU)
model_df

,Op Type,Dimension,Bound,C/M ratio,Op Intensity,Latency (msec),Cycles,C Effcy,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Throughput (Tflops),Compute time (msec),Memory time (msec),Compute cycle,Memory cycle
0,GEMM,"[1, 12288, 1, 4096]",M,0.004294,0.999675,0.097688,91826.756795,0.8,100.663296,0.007812,96.0,0.023438,96.03125,1.030457,0.000419,0.097688,394.264576,91826.756795
1,Logit,"[1, 32, 1, 1024, 128, 32]",M,0.004258,0.991288,0.00821,7716.973623,0.8,8.388608,0.007812,8.0,0.0625,8.070312,1.021811,0.000035,0.00821,32.855381,7716.973623
2,Logit,"[1, 32, 1, 32, 128, 32]",M,0.004134,0.962406,0.000264,248.392423,0.8,0.262144,0.007812,0.25,0.001953,0.259766,0.992041,0.000001,0.000264,1.026731,248.392423
3,Attend,"[1, 32, 1, 1024, 128, 32]",M,0.004258,0.991288,0.00821,7716.973623,0.8,8.388608,0.0625,8.0,0.007812,8.070312,1.021811,0.000035,0.00821,32.855381,7716.973623
4,Attend,"[1, 32, 1, 32, 128, 32]",M,0.004134,0.962406,0.000264,248.392423,0.8,0.262144,0.001953,0.25,0.007812,0.259766,0.992041,0.000001,0.000264,1.026731,248.392423
5,GEMM,"[1, 4096, 1, 4096]",M,0.004293,0.999512,0.032568,30613.899231,0.8,33.554432,0.007812,32.0,0.007812,32.015625,1.030289,0.00014,0.032568,131.421525,30613.899231
6,GEMM,"[1, 22016, 1, 4096]",M,0.004294,0.999711,0.175018,164517.025153,0.8,180.355072,0.007812,172.0,0.041992,172.049805,1.030494,0.000751,0.175018,706.390699,164517.025153
7,GEMM,"[1, 4096, 1, 11008]",M,0.004294,0.999665,0.087513,82262.247801,0.8,90.177536,0.020996,86.0,0.007812,86.028809,1.030447,0.000376,0.087513,353.195349,82262.247801


In [ ]:
model_df.to_csv("llama_7b_decode.csv", index=False)

In [ ]:
llama_summary = get_summary_table(model_df, system=TPU)
llama_summary

,MACs (MFLOP),Total Data (MB),Total Weights (MB),Parameters (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles
0,422.05184,402.785645,386.0,386.0,172.049805,0.409735,385150.661071
